<a href="https://colab.research.google.com/github/armandordorica/MIE1516_A1_Variable_Elimination/blob/master/MIE1516_A1_Q3_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PrettyTable

In [0]:
import prettytable
import numpy as np
pretty=prettytable.PrettyTable()

In [0]:
class MyBayesianModel: 
  def __init__(self, list_of_edges):
    self.list_of_edges = list(list_of_edges)
    self.tabular_cpds=[]

  def get_variables(self):
      list_of_tuples = list(self.list_of_edges)
      list_of_items = [item for t in list_of_tuples for item in t] 
      list_set = set(list_of_items) 
      # convert the set to the list 
      unique_list_of_vars = (list(list_set))
      print(unique_list_of_vars)

  def available_cpds(self):
    for i in range(0, len(self.tabular_cpds)):
      self.tabular_cpds[i].print_factor()


In [0]:
class Factor:
  def __init__(self, indep_var, dep_vars=[]):
    self.indep_var = indep_var
    self.dep_vars = dep_vars
  
  def print_factor(self):

    if len(self.dep_vars)>0:
      self.dep_vars = set(self.dep_vars)
      self.dep_vars = list(self.dep_vars)
      self.dep_vars.sort()
      #print(self.dep_vars)
      dep_vars = str(self.dep_vars[0])
      for i in range (1, len(self.dep_vars)):
        dep_vars = dep_vars + "," + self.dep_vars[i]
      print("P({}|{})".format(self.indep_var, dep_vars))
    if len(self.dep_vars)==0:
      print("P({})".format(self.indep_var))

  def vars_in_factor(self):
    factors = list()
    factors.append(str(self.indep_var))
    for i in range (0, len(self.dep_vars)):
      factors.append(self.dep_vars[i])
    
    list_set = set(factors) 
    # convert the set to the list 
    unique_list_of_vars = list(list_set)
    unique_list_of_vars.sort(reverse=False)
    # print(unique_list_of_vars)
    return unique_list_of_vars

  def contains_var(self, variable):
    if variable in self.vars_in_factor():
      return True
    else:
      return False

  

In [0]:
class TabularCPD: 
  def __init__(self, variable, variable_card, values, evidence='', evidence_card=''): 
    self.variable = variable 
    self.variable_card = variable_card
    self.values = values
    self.evidence = evidence
    self.evidence_card = evidence_card

  # Initializing factors of the CPD depending on the format (whether evidence is provided or not)
    if len(self.evidence)>0:
      self.factors = []
      self.factors.append(Factor(self.variable, self.evidence))
    
    if len(self.evidence)==0:
      self.factors = []
      self.factors.append(Factor(self.variable))
    
    model.tabular_cpds.append(self)
  def print_cpd(self):
    if len(self.evidence) > 0:
      print("P({}|{})".format(self.variable, self.evidence[0]))
      pretty=prettytable.PrettyTable()
      pretty.add_column("{}".format(self.variable), ["{}(0)".format(self.variable),"{}(1)".format(self.variable)], align='l',valign='t')
      
      self.values = np.array(self.values)
      self.values = self.values.transpose()

      for i in range(0,len(self.values)):
        pretty.add_column("{}({})".format(self.evidence[0], i), self.values[i], align='l',valign='t')   
      print(pretty)

    else: 
      print("P({})".format(self.variable))
      pretty=prettytable.PrettyTable()
      pretty.add_column("{}".format(self.variable), ["{}(0)".format(self.variable),"{}(1)".format(self.variable)], align='l',valign='t')
      pretty.add_column("Pr", [item for t in self.values for item in t], align='l',valign='t')
      print(pretty)

  def get_factor(self):
    return self.factors[0]
    
  def print_factor(self):
    self.factors[0].print_factor()

In [0]:
  def eliminate_variable(list_of_factors, variable_to_eliminate):
    print("Marginalize on {}".format(variable_to_eliminate))
    print("Multiply factors:")
    for i in range(0,len(list_of_factors)):
      if list_of_factors[i].get_factor().contains_var(variable_to_eliminate):
        list_of_factors[i].print_factor()

    print("CPDs are:\n")
    for i in range(0,len(list_of_factors)):
      if list_of_factors[i].get_factor().contains_var(variable_to_eliminate):
        list_of_factors[i].print_cpd()

In [0]:
model = MyBayesianModel([('C', 'B'), ('B', 'A')])
cpd_c = TabularCPD(variable='C', variable_card = 2, values=[[0.8,0.2]])
cpd_a = TabularCPD(variable='A', variable_card=2, 
                   values=[[0.3, 0.9], 
                           [0.7, 0.1]],
                   evidence = ['B'], 
                   evidence_card=[2] 
                   )

cpd_b = TabularCPD(variable='B', variable_card=2, 
                   values=[[0.5, 0.4], 
                           [0.5, 0.6]],
                   evidence = ['C'], 
                   evidence_card=[2] 
                   )

In [289]:
eliminate_variable(model.tabular_cpds, "B")

Marginalize on B
Multiply factors:
P(A|B)
P(B|C)
CPDs are:

P(A|B)
+------+------+------+
| A    | B(0) | B(1) |
+------+------+------+
| A(0) | 0.3  | 0.9  |
| A(1) | 0.7  | 0.1  |
+------+------+------+
P(B|C)
+------+------+------+
| B    | C(0) | C(1) |
+------+------+------+
| B(0) | 0.5  | 0.4  |
| B(1) | 0.5  | 0.6  |
+------+------+------+
